# A Toolbox for Solving Dynamic Rational Inattention Problems


This notebook goes through a couple of examples for solving LQG Dynamic Rational Inattention models using the methods and the [solver](https://github.com/choongryulyang/dynamic_multivariate_RI) from [Afrouzi and Yang (2019)](http://www.afrouzi.com/dynamic_inattention.pdf) that was adapted for Julia by [Miguel Acosta](https://www.acostamiguel.com/home). To include the solver, execute:

In [1]:
include("Solve_RI_Dynamics.jl");

## Example 4: Pricing in GE
### Background
Suppose now that there is general equilibirum feedback with the degree of strategic complementarity $\alpha$: $$p_{i,t}^*=(1-\alpha)q_t+\alpha p_t$$ where
\begin{align}
    \Delta q_t&=\rho \Delta q_{t-1}+u_t,\quad u_t\sim \mathcal{N}(0,\sigma_u^2) \\
    p_t&\equiv \int_0^1 p_{i,t}di
\end{align}
Note that now the state space representation for $p_{i,t}^*$ is no longer exogenous and is determined in the equilibrium. Moreover, in the particular case where there is no cost to attention, we know that in the equilibrium $p_{i,t}^*=q_t$. 

Moreover, for the general case, we know that $p_{i,t}^*$ is a Guassian process and depends on the history of shocks to $q_t$. Accordingly, we will approximate it using the following representation:

$$p_{i,t}^*\approx q_t+\sum_{i=0}^{l_\rho-1} \rho_i \Delta q_{t-i}+\sum_{k=0}^{l_u-1} \theta_i u_{t-k}$$

Note that with no inattention, this approximation is accurate with all $\rho_i$'s and $\theta_i$'s begin zero. Our objective is to find the equilibrium values of $\rho_i$'s and $\theta_i$'s and we choose $l_\rho,l_u$ such that this approximation is accurate enough subject to having a managable number of endogenous objects to solve for. Larger $l_\rho,l_u$ are always better for accuracy, but decrease computational efficiency of the solver by increasing the number of states. 

Now, let 
$$\vec{x}_t = (q_{t},\Delta q_{t},\dots,\Delta q_{t-l_{\rho}+1},u_{t},\dots,u_{t-l_u+1})'\in\mathbb{R}^{l_{\rho}+l_u+1}$$

### Matrix Notation

We have
$$\vec{x}_t=\mathbf{A}\vec{x}_{t-1}+\mathbf{Q}u_t$$ where

\begin{align}
    \mathbf{A}=\left[\begin{array}{ccccc}
        1 &\rho| &&0&\\ \hline
        0 &\rho| &&0&\\ \hline
        0| & \mathbf{I}_{(l_{\rho}-1)\times(l_{\rho}-1)}&|  & \mathbf{0}_{(l_{\rho}-1)\times(l_{u}+1)} & \\ \hline
         &  & \mathbf{0}_{1\times(l_{\rho}+l_{u}+1)}& & \\ \hline
         & \mathbf{0}_{(l_{u}-1)\times(l_{\rho}+2)} &| & \mathbf{I}_{(l_{u}-1)\times(l_{u}-1)} & 
        \end{array}\right], \quad 
    \mathbf{Q}=\left[\begin{array}{c}
         \sigma_u \\
         \sigma_u \\ \hline
         \mathbf{0}_{(l_{\rho}-1)\times 1} \\ \hline
         1 \\ \hline
         \mathbf{0}_{(l_{u}-1)\times 1} 
        \end{array}\right]
\end{align}
and 
\begin{align}
    p_{i,t}^*=\Gamma'\vec{x}_t
\end{align}

Our objective is to find the equilibrium $\Gamma$, and we do this by iteration for an initial guess $\Gamma_{(0)}$:

* At iteration $n\geq 1$, using function solve the ratinoal inattention problem given the guess $\Gamma_{(n-1)}$.

* Get the implied impulse response of $p_{i,t^*}-q_{t}$ and approximate it to fit the state-space represtation.

* Update $\Gamma_{(n)}$.
* Iterate until convergence.

## Setup the Problem

In [27]:
function approx(Hpq,   # irf of p*-q: p*-q = Hpq'U
				Hdq,   # irf of dq:   dq   = Hdq'U
				l_ρ,   # number of dq lags in the approx
				l_u)   # number of u lags in the approx
	T = length(Hpq);
    ### Add the error for if length(Hdq)~=length(Hpq)
    ### Add the error for l_u = 0 or l_ρ = 0 
	
	M = [zeros(1,T-1) 0; Matrix(I,T-1,T-1) zeros(T-1,1)]
    B = zeros(l_ρ+l_u,T);

    for i in 1:l_ρ
        B[i,:] = Hdq'*M'^(i-1);
    end

    B[l_ρ+1:l_ρ+l_u,1:l_u] = Matrix(I,l_u,l_u); 
    Γ = [1;inv(B*B')*B*Hpq];
    return(Γ)
end
    
function ge_drip(ω,β,A,Q,                     #primitives of drip except for H which is endogenous
                 α,                           #strategic complementarity
                 l_ρ,l_u,                     #parameters of ARMA(p,q)
                 Hdq;                         #irf of Δq
                 Γ0    = [1;zeros(l_ρ+l_u)],  #optional: initial guess for Γ0
                 maxit = 200,                 #optional: max number of iterations for GE code 
                 tol   = 1e-12)               #optional: tolerance for iterations

    # set primitives
    T     = length(Hq);
    err   = 1;  
    iter  = 0;
    M     = [zeros(1,T-1) 0; Matrix(I,T-1,T-1) zeros(T-1,1)];
    eye   = Matrix(I,T,T);
    
    # iterate on GE
    while (err > tol) & (iter < maxit)
            if iter == 0 
                global ge  = Solve_RI_Dynamics(ω,β,A,Q,Γ0); 
            else 
                global ge  = Solve_RI_Dynamics(ω,β,A,Q,Γ0;Ω0 = ge.Ω ,Σ0 = ge.Σ_1);
            end
            K = real.(ge.K)
            Y = real.(ge.Y)

             if maximum(abs.(imag.(ge.K))) > 1e-6 || maximum(abs.(imag.(ge.Y))) > 1e-6
                 println("Complex components are large")
             end

            irfs  = dripirfs(ge,T);
            Hpq   = α*(irfs.a[1,1,:]-inv(I-M)*Hdq);
            Γ1    = approx(Hpq,Hdq,l_ρ,l_u);
            err   = norm(Γ1-Γ0,2);
        
            Γ0    = Γ1;
            if iter == maxit
                print("GE loop hit maxit -- no convergence\n")
            end
            iter += 1;
            println("Iteration $iter. Difference: $err")
    end
    return(ge)
end     

ge_drip (generic function with 1 method)

In [22]:
## Parameters
ρ   = 0.6;
σ_u = 0.1;
α   = 0.8;
ω   = 0.4;
β   = 0.96^0.25;
Γ0  = zeros(1+l_ρ+l_u,1); Γ0[1]=1;
T   = 60;
Hdq = ρ.^(0:T-1); #state-space rep. of Δq 

l_ρ = 1;
l_u = 50;
A   = [[1; ρ; zeros(l_ρ+l_u-1,1)]';
	  [0; ρ; zeros(l_ρ+l_u-1,1)]';
	  [zeros(1,l_ρ-1); Matrix(I,l_ρ-1,l_ρ-1); zeros(l_u+1,l_ρ-1)]';
	   zeros(1,l_ρ+l_u+1);
	  [zeros(l_ρ+2,l_u-1); Matrix(I,l_u-1,l_u-1)]'];
Q   = [σ_u;σ_u;zeros(l_ρ-1,1);1;zeros(l_u-1,1)];

In [23]:
ex4 = ge_drip(ω,β,A,Q,α,l_ρ,l_u,Hdq)

Iteration 1. Difference: 176216.1084739109
Iteration 2. Difference: 4596.463637287462
Your matrix has complex stuff

MethodError: MethodError: no method matching abs(::Nothing)
Closest candidates are:
  abs(!Matched::Pkg.Resolve.MaxSum.FieldValues.FieldValue) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/Pkg/src/resolve/FieldValues.jl:67
  abs(!Matched::Pkg.Resolve.VersionWeights.VersionWeight) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/Pkg/src/resolve/VersionWeights.jl:40
  abs(!Matched::Missing) at missing.jl:100
  ...

In [35]:
x = approx(Hdq,Hdq,5,5)   

11-element Array{Float64,1}:
  1.0                  
 -0.10845141164655368  
  0.07187909711922837  
  0.298025528943023    
  0.01380500966546653  
 -0.4410936702173714   
  1.0046786502120397   
  0.9247991534196831   
 -0.13138785448581103  
  0.809816143591199    
  6.174103337471289e-17